<a href="https://colab.research.google.com/github/zusmoshijiku/parabrisas-OCR/blob/main/BarrientosJose_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# -*- coding: utf-8 -*-
"""BarrientosJoseT1"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag

# A Funcion la cual entrega los esfuerzos internos de un elemento estructural

def ElementInternalForces2D(xeset, xloc, s_i, q_):
    n = 100
    elem_L = np.linalg.norm(xeset[1] - xeset[0])
    nhat = (xeset[1] - xeset[0]) / elem_L
    Q = np.array([[nhat[0], nhat[1]], [-nhat[1], nhat[0]]])
    av = block_diag(Q, 1)
    q1, q2 = Q @ q_
    Ne = np.zeros(n)
    Ve = np.zeros(n)
    Me = np.zeros(n)
    S1, S2, S3 = av @ s_i
    for x in range(n):
        Ne[x] = -S1 - np.trapz(q1[:x+1], xloc[:x+1])
        Ve[x] = -S2 - np.trapz(q2[:x+1], xloc[:x+1])
        Me[x] = -S3 + S2 * xloc[x] + np.trapz((xloc[x] - xloc[:x+1]) * q2[:x+1], xloc[:x+1])

    if nhat[0] < 0 or (nhat[0] == 0 and nhat[1] < 0):
            Me = -Me
    return Ne, Ve, Me

#B Graficar la envolvente de un esfuerzo interno de uno de los elementos

def PlotElementDiagram2D(xeset, xloc, ydiagram, scale, **kwargs):

    plt.figure(figsize=(9, 9))
    resta_nodos_elem = xeset[1] - xeset[0]
    elem_L = np.linalg.norm(resta_nodos_elem)
    nhat = resta_nodos_elem / elem_L
    Q = np.array([[nhat[0], nhat[1]],
                 [-nhat[1], nhat[0]]])

    Xe = np.hstack((xloc, np.array((xloc[-1], xloc[0], xloc[0]))))

    if 'Ne' in kwargs and np.array_equal(ydiagram, kwargs['Ne']):
        color = "blue"
        title = f'Esfuerzo Axial del Elemento (escala {scale})'
        Ye = np.hstack((ydiagram, np.array((0, 0, ydiagram[0]))))
    elif 'Ve' in kwargs and np.array_equal(ydiagram, kwargs['Ve']):
        color = "green"
        title = f'Esfuerzo de Corte del Elemento (escala {scale})'
        Ye = np.hstack((ydiagram, np.array((0, 0, ydiagram[0]))))
    elif 'Me' in kwargs and np.array_equal(ydiagram, kwargs['Me']):
        color = "red"
        title = f'Momento Flector del Elemento (escala {scale})'
        Ye = - np.hstack((ydiagram, np.array((0, 0, ydiagram[0]))))
    xyz = Q.T @ np.vstack((Xe, Ye * scale))
    xyz[0] += xeset[0, 0]
    xyz[1] += xeset[0, 1]

    plt.xlabel("Longitud del Elemento X (m)")
    plt.ylabel("Longitud del Elemento Y (m)")

    ydiagram_max = np.max(ydiagram)
    ydiagram_min = np.min(ydiagram)
    xind_max = np.argmax(ydiagram)
    xind_min = np.argmin(ydiagram)
    x_max, y_max = xyz[0][xind_max], xyz[1][xind_max]
    x_min, y_min = xyz[0][xind_min], xyz[1][xind_min]
    plt.scatter([x_max], [y_max], color="blue", s=100, label=f"Máximo: {ydiagram_max:.2f}")
    plt.scatter([x_min], [y_min], color="green", s=100, label=f"Mínimo: {ydiagram_min:.2f}")
    props = dict(boxstyle='round', facecolor='white', alpha=0.8)
    plt.text(x_max, y_max, f"{ydiagram_max:.2f}", fontsize=12, color="blue", ha="left", va="bottom", bbox=props)
    plt.text(x_min, y_min, f"{ydiagram_min:.2f}", fontsize=12, color="green", ha="left", va="bottom", bbox=props)
    plt.title(title)

    plt.fill(xyz[0], xyz[1], facecolor=color, edgecolor='black', lw=1, alpha=0.3)

    plt.plot(xeset.T[0], xeset.T[1], 'k-', linewidth=2)
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.legend()
    plt.axis('equal')
    plt.show()

#D
def PlotDiagram2D(NODE, ELEM, XLoc, YDiagram, scale):
    cant_elem = len(ELEM)
    plt.figure(figsize=(9, 9))

    plt.xlabel("Coordenada X (m)")
    plt.ylabel("Coodenada Y (m)")
    for i in range(cant_elem):
        nodoi_elem = int(ELEM[i,0])
        nodof_elem = int(ELEM[i,1])
        coords_nodoi = NODE[nodoi_elem]
        coords_nodof = NODE[nodof_elem]
        plt.plot([coords_nodoi[0], coords_nodof[0]], [coords_nodoi[1], coords_nodof[1]], color='k', lw=3, zorder=-1)
        elem_L = np.linalg.norm(coords_nodof - coords_nodoi)
        nhat = (coords_nodof - coords_nodoi)/elem_L
        x_loc = XLoc[i]
        y_diagram = YDiagram[i]
        Q  = np.array([[nhat[0],nhat[1]],
                    [-nhat[1],nhat[0]]])
        Xe = np.hstack((x_loc, np.array((x_loc[-1], x_loc[0], x_loc[0]))))
        Ye = np.hstack((y_diagram, np.array((0, 0, y_diagram[0]))))
        xyz = Q.T @ np.vstack((Xe, Ye*scale))
        xyz[0] += coords_nodoi[0]
        xyz[1] += coords_nodoi[1]
        ydiagram_max = np.max(y_diagram)
        ydiagram_min = np.min(y_diagram)
        xind_max = np.argmax(y_diagram)
        xind_min = np.argmin(y_diagram)
        x_max, y_max = xyz[0][xind_max], xyz[1][xind_max]
        x_min, y_min = xyz[0][xind_min], xyz[1][xind_min]
        plt.scatter([x_max], [y_max], color="blue", s=100, label=f"Máximo: {ydiagram_max}")
        plt.scatter([x_min], [y_min], color="green", s=100, label=f"Mínimo: {ydiagram_min}")
        plt.text(x_max, y_max, f"{round(ydiagram_max,2)}", fontsize=11, color="blue", ha="left", va="bottom")
        plt.text(x_min, y_min, f"{round(ydiagram_min,2)}", fontsize=11, color="green", ha="left", va="bottom")
        plt.fill(xyz[0], xyz[1], facecolor="cyan", edgecolor='black', lw=1, alpha=0.3)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()

#F Funcion para los esfuerzos internos de la estructura

def InternalForces2D(NODE, ELEM, XLoc, Si, Qx_, Qy_):
    n = 100
    cant_elem = len(ELEM)
    M = np.zeros((cant_elem,n))
    V = np.zeros((cant_elem,n))
    N = np.zeros((cant_elem,n))
    for j in range(cant_elem):
        nodoi_elem = int(ELEM[j,0])
        nodof_elem = int(ELEM[j,1])
        coords_nodoi = NODE[nodoi_elem]
        coords_nodof = NODE[nodof_elem]
        xeset_ = np.vstack((coords_nodoi, coords_nodof))
        s_i = Si[j]
        q_ = np.array([Qx_[j], Qy_[j]])
        x_loc = XLoc[j]
        Ne, Ve, Me = ElementInternalForces2D(xeset_, x_loc, s_i, q_)
        N[j] = Ne
        V[j] = Ve
        M[j] = Me
    return N, V, M